In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import keras.layers as lay
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Constants

In [2]:
batch_size = 32
opt = [keras.optimizers.Adam(1e-4,.5),keras.optimizers.Adam(1e-4,.5)]
bce = keras.losses.BinaryCrossentropy()

# Generator

In [3]:
class Fade_in(keras.Layer):
    def __init__(self, alpha=0.0, activity_regularizer=None, trainable=True, dtype=None, autocast=True, name=None, **kwargs):
        super().__init__(activity_regularizer=activity_regularizer, trainable=trainable, dtype=dtype, autocast=autocast, name=name, **kwargs)
        self.alpha = tf.Variable(alpha,trainable=False)

    def call(self,inputs):
        antigo, novo = inputs
        return self.alpha*novo + (1-self.alpha)*antigo
        

def g_block(x, filters):
    x = lay.UpSampling2D((2,2))(x)
    x = lay.Conv2D(filters,(3,3),(1,1),'same')(x)
    x = lay.LeakyReLU(.2)(x)
    return x


def build_generator_4x4(latent_dim=128):
    inputs = lay.Input((latent_dim,))
    x = inputs
    x = lay.Dense(4*4*512,activation='leaky_relu')(x)
    x = lay.Reshape((4,4,512),name='lastfeat_4x4')(x)
    img_output = lay.Conv2D(3,(1,1),(1,1),'same',activation='tanh',name='toRGB_4x4')(x)
    return keras.Model(inputs,img_output,name='Generator_4x4')

def build_generator_8x8(g_4x4):
    latent_input = g_4x4.input
    g_4x4.trainable = False
    last_featuremap = g_4x4.get_layer('lastfeat_4x4').output
    old_path = lay.UpSampling2D((2,2))(g_4x4.output)
    x = g_block(last_featuremap,512)
    new_path = lay.Conv2D(3,(1,1),(1,1),'same',activation='tanh',name='toRGB_8x8')(x)
    output_fade_in = Fade_in(name='fade_8')([old_path,new_path])
    return keras.Model(latent_input,output_fade_in,name='Generator_8x8')

def build_generator_16x16(g_8x8):
    latent_input = g_8x8.input
    g_8x8.trainable = False
    last_featuremap = g_8x8.layers[-4].output
    old_path = lay.UpSampling2D((2,2))(g_8x8.output)
    x = g_block(last_featuremap,512)
    new_path = lay.Conv2D(3,(1,1),(1,1),'same',activation='tanh',name='toRGB_16x16')(x)
    output_fade_in = Fade_in(name='fade_16')([old_path,new_path])
    return keras.Model(latent_input,output_fade_in,name='Generator_16x16')

def build_generator_32x32(g_16x16):
    latent_input = g_16x16.input
    g_16x16.trainable = False
    last_featuremap = g_16x16.layers[-4].output
    old_path = lay.UpSampling2D((2,2))(g_16x16.output)
    x = g_block(last_featuremap,512)
    new_path = lay.Conv2D(3,(1,1),(1,1),'same',activation='tanh',name='toRGB_32x32')(x)
    output_fade_in = Fade_in(name='fade_32')([old_path,new_path])
    return keras.Model(latent_input,output_fade_in,name='Generator_32x32')

# Discriminator

In [4]:
def d_block(x,filters,name):
    x = lay.Conv2D(filters,(3,3),(1,1),'same',activation='leaky_relu',name=f'conv{name}_1')(x)
    x = lay.Conv2D(filters,(3,3),(1,1),'same',activation='leaky_relu',name=f'conv{name}_2')(x)
    return x

def build_discriminator_4x4(filters=128):
    inputs = lay.Input((4,4,3))
    x = inputs
    x = lay.Conv2D(128,(3,3),(1,1),'same',activation='leaky_relu',name='fromRGB_4x4')(x)
    x = lay.Conv2D(filters,(3,3),(1,1),'same',activation='leaky_relu',name='conv4_1')(x)
    x = lay.Conv2D(filters,(3,3),(1,1),'same',activation='leaky_relu',name='conv4_2')(x)
    x = lay.Flatten(name='flatten')(x)
    x = lay.Dense(1,activation='sigmoid',name='brain')(x)
    return keras.Model(inputs,x)

def build_discriminator_8x8(d_4x4, filters=128):
    inputs = lay.Input((8,8,3))

    # Caminho Novo
    x = inputs
    x = lay.Conv2D(128,(3,3),(1,1),'same',activation='leaky_relu',name='fromRGB_8x8')(x)
    x = d_block(x,128,8)
    new_path = lay.AvgPool2D((2,2),name='pool8_1')(x)

    # Caminho antigo
    x = inputs
    x = lay.AvgPool2D((2,2))(x)
    old_path = d_4x4.layers[1](x)

    # União dos caminhos
    output_fade_in = Fade_in(name='fade_8')([old_path,new_path])
    x = output_fade_in

    # for i in range(2,len(d_4x4.layers)):
    #     x = d_4x4.layers[i](x)
    for i in range(2,len(d_4x4.layers)):
        x = d_4x4.layers[i](x)
    
    return keras.Model(inputs,x)

def build_discriminator_16x16(d_8x8, filters=128):
    inputs = lay.Input((16,16,3))

    # Caminho Novo
    x = inputs
    x = lay.Conv2D(128,(3,3),(1,1),'same',activation='leaky_relu',name='fromRGB_16x16')(x)
    x = d_block(x,128,16)
    new_path = lay.AvgPool2D((2,2),name='pool16_1')(x) # shape (None, 8, 8, 128)

    # Caminho antigo
    x = inputs
    x = lay.AvgPool2D((2,2))(x) # shape (None, 8, 8, 3)
    old_path = d_8x8.layers[1](x)

    # União dos caminhos
    output_fade_in = Fade_in(name='fade_16')([old_path,new_path])
    x = output_fade_in

    path = ['conv8_1','conv8_2','pool8_1','conv4_1','conv4_2','flatten','brain']
    for name in path:
        x = d_8x8.get_layer(name)(x)
    
    return keras.Model(inputs,x)

def build_discriminator_32x32(d_16x16, filters=128):
    inputs = lay.Input((32,32,3))

    # Caminho Novo
    x = inputs
    x = lay.Conv2D(128,(3,3),(1,1),'same',activation='leaky_relu',name='fromRGB_32x32')(x)
    x = d_block(x,128,32)
    new_path = lay.AvgPool2D((2,2),name='pool32_1')(x) # shape (None, 8, 8, 128)

    # Caminho antigo
    x = inputs
    x = lay.AvgPool2D((2,2))(x) # shape (None, 8, 8, 3)
    old_path = d_16x16.layers[1](x)

    # União dos caminhos
    output_fade_in = Fade_in(name='fade_32')([old_path,new_path])
    x = output_fade_in

    path = ['conv16_1','conv16_2','pool16_1','conv8_1','conv8_2','pool8_1','conv4_1','conv4_2','flatten','brain']
    for name in path:
        x = d_16x16.get_layer(name)(x)
    
    return keras.Model(inputs,x)



In [5]:
g_4x4 = build_generator_4x4()
# g_8x8 = build_generator_8x8(g_4x4)
# g_16x16 = build_generator_16x16(g_8x8)
# g_32x32 = build_generator_32x32(g_16x16)

In [6]:
d_4x4 = build_discriminator_4x4(filters=512)
# d_8x8 = build_discriminator_8x8(d_4x4,128)
# d_16x16 = build_discriminator_16x16(d_8x8,128)
# d_32x32 = build_discriminator_32x32(d_16x16,128)

# Função de passo de treinamento

In [7]:
@tf.function
def train_step(gan,batch,opt,bce):
    g_loss = 0.
    d_loss = 0.
    batch_size = tf.shape(batch)[0]
    noise = tf.random.normal((2*batch_size,128),0,1)
    with tf.GradientTape() as tape:
        fake_imgs = gan[0](noise, training=True)
        fake_logits = gan[1](fake_imgs, training=True)
        g_loss = bce(tf.ones_like(fake_logits),fake_logits)
    grads = tape.gradient(g_loss,gan[0].trainable_variables)
    opt[0].apply_gradients(zip(grads,gan[0].trainable_variables))

    noise = tf.random.normal((batch_size,128),0,1)
    with tf.GradientTape() as tape:
        fake_imgs = gan[0](noise,training=True)
        fake_logits = gan[1](fake_imgs,training=True)
        true_logits = gan[1](batch,training=True)
        d_loss += bce(tf.zeros_like(fake_logits),fake_logits) + bce(tf.ones_like(true_logits),true_logits)
        d_loss /= 2
    grads = tape.gradient(d_loss,gan[1].trainable_variables)
    opt[1].apply_gradients(zip(grads,gan[1].trainable_variables))
    return g_loss, d_loss

# Dataset

In [8]:
(x,y),(_,_) = keras.datasets.cifar10.load_data()
x = x[y[:,0]==1]
x = (tf.cast(x,tf.float32)-127.5)/127.5
x = tf.image.resize(x,(4,4))
train_4x4 = tf.data.Dataset.from_tensor_slices((tf.image.resize(x,(4,4)))).cache().shuffle(1024).batch(batch_size,drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [9]:
for batch in train_4x4:
    g_loss, d_loss = train_step([g_4x4,d_4x4],batch,opt,bce)
    print(f'G: {g_loss}; D: {d_loss}')

G: 0.6944639682769775; D: 0.6940869092941284
G: 0.684400200843811; D: 0.6743226647377014
G: 0.6763319969177246; D: 0.6546305418014526
G: 0.665186882019043; D: 0.6353511810302734
G: 0.6562385559082031; D: 0.6183775663375854
G: 0.6433091759681702; D: 0.609485924243927
G: 0.6262623071670532; D: 0.5878362655639648
G: 0.6077922582626343; D: 0.5826013088226318
G: 0.589589536190033; D: 0.5574988722801208
G: 0.5861729383468628; D: 0.5718101263046265
G: 0.5753055214881897; D: 0.5612285137176514
G: 0.5624241828918457; D: 0.5629803538322449
G: 0.5575814247131348; D: 0.5561191439628601
G: 0.5567529201507568; D: 0.5582883358001709
G: 0.5793899893760681; D: 0.5828688144683838
G: 0.5862235426902771; D: 0.5363423824310303
G: 0.5803622007369995; D: 0.5644469261169434
G: 0.5885434150695801; D: 0.5654316544532776
G: 0.5907149314880371; D: 0.5457075834274292
G: 0.5975813865661621; D: 0.512597382068634
G: 0.5779749155044556; D: 0.5453053116798401
G: 0.604117751121521; D: 0.5565358996391296
G: 0.60617470741

2025-08-29 10:31:21.903267: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
